Спостерiгається система, яка генерує послiдовнiсть незалежних нормально розподiлених випад-
кових величин x1, . . . , xn з дисперсiєю 1. Система може перебувати в одному з чотирьох станiв

k ∈ {0, 1, 2, 3}. Математичне сподiвання кожної величини залежить вiд поточного стану системи
i дорiвнює ak = k коли система знаходиться в в станi k, k ∈ {0, 1, 2, 3}. Ймовiрнiсть того, що система
перебуває в станi k дорiвнює k+1

10 , k ∈ {0, 1, 2, 3}.

Завдання.

1. Згенерувати послiдовнiсть n незалежних нормально розподiлених випадкових величин з дис-
персiєю 1 i математичним сподiванням, яке з ймовiрнiстю k+1

10 дорiвнює k, k ∈ {0, 1, 2, 3}.
2. За допомогою алгоритму самонавчання отримати оцiнки ймовiрностей pK(k) i параметрiв ak,

k ∈ {0, 1, 2, 3}. Умовою зупинки алгоритму вважати наступну: оцiнки параметрiв не змiни-
лись, а оцiнки ймовiрностей змiнились менше нiж на 0.001. Алгоритм має працювати для

довiльного n.
3. Проаналiзувати поведiнку алгоритму в залежностi вiд n i початкових оцiнок ймовiрностей i
параметрiв.

In [74]:
import numpy as np
from scipy.stats import norm

In [20]:
import numpy as np
from scipy.stats import norm


def selflearning_algorithm(
    real_a, real_p, datasize, data, tolerance=1e-3, max_iter=20_000
):
    # Initialization
    print("--" * 20)
    print(f"Data size: {datasize}\n")
    a_k, p_k = (
        np.linspace(min(data), max(data), 4),  # Initialize a_k evenly spaced
        np.ones(shape=4) / 4,  # Initialize p_k uniformly
    )

    print(f"Init a_k: {a_k}")
    print(f"Init p_k: {p_k}")

    for iteration in range(max_iter):
        # E-step: Calculate responsibilities
        gamma = np.zeros((datasize, 4))
        for k in range(4):
            gamma[:, k] = p_k[k] * norm.pdf(data, loc=a_k[k], scale=1)
        gamma /= np.maximum(
            gamma.sum(axis=1, keepdims=True), 1e-10
        )  # Avoid division by zero

        # M-step: Update parameters
        new_a_k = np.sum(gamma * data[:, np.newaxis], axis=0) / (
            gamma.sum(axis=0) + 1e-10
        )
        new_p_k = gamma.sum(axis=0) / datasize

        # Stopping condition
        if np.all(np.abs(new_p_k - p_k) < 1e-5) and np.all(
            np.abs(new_a_k - a_k) < tolerance
        ):
            break

        # Update parameters for next iteration
        p_k, a_k = new_p_k, new_a_k

    print(f"Iteration {iteration + 1} completed")
    print(f"Real a_k: {real_a}")
    print(f"Real p_k: {real_p}")
    print("\n")
    print(f"Final a_k: {a_k.round(2)}")
    print(f"Final p_k: {p_k.round(2)}")
    print(f"Diff:\na_k: {np.abs(real_a - a_k)}\np_k: {np.abs(real_p - p_k)}\n")

    return p_k, a_k


# Parameters for the random variable generation
num_samples = [10, 100, 1000, 10_000, 100_000]  # Number of random samples to generate
mean_options = np.array([0, 1, 2, 3])  # Possible mean values
mean_probabilities = np.array(
    [(k + 1) / 10 for k in mean_options]
)  # Probabilities from task
mean_probabilities /= mean_probabilities.sum()  # Normalize probabilities

# Generation and EM execution
for size in num_samples:
    # Generate sample means according to mean_options and mean_probabilities
    sample_mean = np.random.choice(mean_options, size=size, p=mean_probabilities)
    # Generate normal samples with the chosen means
    normal_samples = np.random.normal(loc=sample_mean, scale=1, size=size)

    print(f"\nGenerated {size} samples\n")
    final_probs, final_means = selflearning_algorithm(
        real_a=mean_options,
        real_p=mean_probabilities,
        datasize=size,
        data=normal_samples,
    )


Generated 10 samples

----------------------------------------
Data size: 10

Init a_k: [0.21522879 1.75210797 3.28898716 4.82586635]
Init p_k: [0.25 0.25 0.25 0.25]
Iteration 26 completed
Real a_k: [0 1 2 3]
Real p_k: [0.1 0.2 0.3 0.4]


Final a_k: [1.44 1.44 4.02 4.03]
Final p_k: [0.27 0.39 0.16 0.18]
Diff:
a_k: [1.44083931 0.4428105  2.02475748 1.02504605]
p_k: [0.16780165 0.19349274 0.13684753 0.22444686]


Generated 100 samples

----------------------------------------
Data size: 100

Init a_k: [-2.06756649  0.60608458  3.27973565  5.95338673]
Init p_k: [0.25 0.25 0.25 0.25]
Iteration 49 completed
Real a_k: [0 1 2 3]
Real p_k: [0.1 0.2 0.3 0.4]


Final a_k: [-1.49  0.8   2.92  5.41]
Final p_k: [0.05 0.43 0.51 0.01]
Diff:
a_k: [1.49032993 0.2042202  0.91768292 2.41143133]
p_k: [0.05324281 0.23020273 0.21359736 0.39055728]


Generated 1000 samples

----------------------------------------
Data size: 1000

Init a_k: [-2.33908238  0.37613532  3.09135301  5.80657071]
Init p_k: [0.25 0

# Висновки щодо поведінки алгоритму

## 1. Залежність точності результатів від розміру вибірки $ n $
- **Малі вибірки ($ n = 10 $, $ n = 100 $)**:
  - Алгоритм демонструє значні розбіжності між реальними значеннями параметрів $ a_k $ та $ p_k $ і оціненими.
  - Через недостатню кількість даних модель не може точно оцінити параметри компонент. 
  - Наприклад:
    - Для $ n = 10 $: значення $ a_k $ і $ p_k $ суттєво відрізняються від реальних, оскільки компоненти недостатньо представлені.
    - Для $ n = 100 $: оцінки покращуються, але все ще спостерігається значне зміщення, особливо для крайніх компонент.

- **Збільшені вибірки ($ n = 1000 $, $ n = 10.000 $, $ n = 100.000 $)**:
  - Зі збільшенням розміру вибірки оцінки $ a_k $ та $ p_k $ стають ближчими до реальних значень.
  - Однак алгоритм продовжує завищувати ймовірності для компонент з більшими кластерами ($ p_k $), тоді як компоненти з малими ймовірностями ($ p_k \approx 0.1 $) систематично недооцінюються.
  - Наприклад:
    - Для $ n = 10.000 $: оцінка $ a_k $ стає ближчою до реальних значень, але $ p_k $ для крайніх компонент залишається неточною.
    - Для $ n = 100.000 $: оцінки $ a_k $ і $ p_k $ стабілізуються, але не досягають повної відповідності через початкові обмеження.

## 2. Залежність від початкових оцінок параметрів
- **Ініціалізація параметрів $ a_k $**:
  - Початкові значення $ a_k $, обрані рівномірно з інтервалу вибірки, можуть бути далекі від реальних центрів кластерів.
  - Для малих вибірок ($ n = 10 $, $ n = 100 $) така ініціалізація призводить до повільної збіжності або значного зміщення оцінок.
  - Для великих вибірок ($ n = 10.000 $, $ n = 100.000 $) ініціалізація стає менш критичною, але все одно впливає на точність кінцевих результатів.